In [1]:
import pandas as pd
import numpy as np
import joblib
import pickle
from mlxtend.frequent_patterns import apriori,association_rules,fpgrowth #unsupervised association
from sklearn.feature_extraction.text import TfidfVectorizer #recommenders
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_csv("Data/data_preprocess.csv")
data

,diseases_abdominal aortic aneurysm,diseases_abdominal hernia,diseases_abscess of nose,diseases_abscess of the lung,diseases_abscess of the pharynx,diseases_acanthosis nigricans,diseases_acariasis,diseases_achalasia,diseases_acne,diseases_actinic keratosis,...,stuttering or stammering,problems with orgasm,nose deformity,lump over jaw,sore in nose,hip weakness,back swelling,ankle stiffness or tightness,ankle weakness,neck weakness
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189642,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
189643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
189644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
189645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
Ap_items = fpgrowth(data,min_support=0.001,use_colnames=True,max_len=5)

In [31]:
Ap_items

,support,itemsets
0,0.085923,(shortness of breath)
1,0.058672,(depressive or psychotic symptoms)
2,0.038319,(chest tightness)
3,0.037607,(anxiety and nervousness)
4,0.024219,(palpitations)
...,...,...
20071,0.001408,"(diseases_conjunctivitis due to virus, swollen..."
20072,0.001371,"(diseases_conjunctivitis due to virus, itchine..."
20073,0.001371,"(diseases_conjunctivitis due to virus, pain in..."
20074,0.001719,"(diminished vision, diseases_conjunctivitis du..."


In [13]:
Ap_items = pd.read_csv("Data/FPgrowthITEMS.csv")

In [14]:
import ast

In [15]:
Ap_items['itemsets'] = Ap_items['itemsets'].apply(
    lambda x: frozenset(ast.literal_eval(x.replace("frozenset(", "").replace(")", "")))
)


In [7]:
diseases = [i for i in data.columns if i.startswith("diseases_")]
symptoms = [i for i in data.columns if not i.startswith("diseases_")]

In [6]:
symptoms

['anxiety and nervousness',
 'depression',
 'shortness of breath',
 'depressive or psychotic symptoms',
 'sharp chest pain',
 'dizziness',
 'insomnia',
 'abnormal involuntary movements',
 'chest tightness',
 'palpitations',
 'irregular heartbeat',
 'breathing fast',
 'hoarse voice',
 'sore throat',
 'difficulty speaking',
 'cough',
 'nasal congestion',
 'throat swelling',
 'diminished hearing',
 'lump in throat',
 'throat feels tight',
 'difficulty in swallowing',
 'skin swelling',
 'retention of urine',
 'groin mass',
 'leg pain',
 'hip pain',
 'suprapubic pain',
 'blood in stool',
 'lack of growth',
 'emotional symptoms',
 'elbow weakness',
 'back weakness',
 'pus in sputum',
 'symptoms of the scrotum and testes',
 'swelling of scrotum',
 'pain in testicles',
 'flatulence',
 'pus draining from ear',
 'jaundice',
 'mass in scrotum',
 'white discharge from eye',
 'irritable infant',
 'abusing alcohol',
 'fainting',
 'hostile behavior',
 'drug abuse',
 'sharp abdominal pain',
 'feeling 

In [206]:
rules = association_rules(Ap_items,metric="confidence",min_threshold=0.001) #includes every possible diseases to increase diversity!

In [207]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(shortness of breath),(headache),0.085923,0.099000,0.008458,0.098435,0.994297,1.0,-0.000049,0.999374,-0.006236,0.047929,-0.000627,0.091934
1,(headache),(shortness of breath),0.099000,0.085923,0.008458,0.085433,0.994297,1.0,-0.000049,0.999464,-0.006326,0.047929,-0.000536,0.091934
2,(shortness of breath),(sharp chest pain),0.085923,0.096780,0.032323,0.376189,3.887061,1.0,0.024008,1.447907,0.812553,0.214944,0.309348,0.355088
3,(sharp chest pain),(shortness of breath),0.096780,0.085923,0.032323,0.333987,3.887061,1.0,0.024008,1.372462,0.822320,0.214944,0.271382,0.355088
4,(cough),(shortness of breath),0.099654,0.085923,0.019320,0.193873,2.256359,1.0,0.010758,1.133912,0.618438,0.116207,0.118097,0.209363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118855,"(lacrimation, diseases_conjunctivitis due to v...",(diminished vision),0.001708,0.033019,0.001060,0.620370,18.788147,1.0,0.001003,2.547169,0.948395,0.031480,0.607407,0.326234
118856,"(diminished vision, diseases_conjunctivitis du...",(lacrimation),0.001719,0.029655,0.001060,0.616564,20.791179,1.0,0.001009,2.530660,0.953542,0.034963,0.604846,0.326152
118857,(lacrimation),"(diminished vision, diseases_conjunctivitis du...",0.029655,0.001719,0.001060,0.035740,20.791179,1.0,0.001009,1.035282,0.980994,0.034963,0.034079,0.326152
118858,(diminished vision),"(lacrimation, diseases_conjunctivitis due to v...",0.033019,0.001708,0.001060,0.032098,18.788147,1.0,0.001003,1.031398,0.979104,0.031480,0.030442,0.326234


In [107]:
joblib.dump(rules,"Models/FPgrowth.pkl")

['Models/FPgrowth.pkl']

#### Model Implementation for Normal Dataset(will be used later if needed)

In [2]:
nrml = pd.read_csv("Data/normalHealth.csv")
nrml

,Disease:(vertigo) Paroymsal Positional Vertigo,Disease:AIDS,Disease:Acne,Disease:Alcoholic hepatitis,Disease:Allergy,Disease:Arthritis,Disease:Bronchial Asthma,Disease:Cervical spondylosis,Disease:Chicken pox,Disease:Chronic cholestasis,...,Symptoms: watering_from_eyes,Symptoms: weakness_in_limbs,Symptoms: weakness_of_one_body_side,Symptoms: weight_gain,Symptoms: weight_loss,Symptoms: yellow_crust_ooze,Symptoms: yellow_urine,Symptoms: yellowing_of_eyes,Symptoms: yellowish_skin,Symptoms:itching
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
300,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
301,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
302,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [10]:
ap_d = [i for i in nrml.columns if i.startswith("Disease:")]
ap_s = [i for i in nrml.columns if i.startswith('Symptoms:')]

In [14]:
apr = fpgrowth(nrml,min_support=0.05,use_colnames=True)
apr

C:\Users\lipun\jupyterfix\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.138158,(Symptoms: skin_rash)
1,0.134868,(Symptoms:itching)
2,0.174342,(Symptoms: chills)
3,0.384868,(Symptoms: vomiting)
4,0.138158,(Symptoms: chest_pain)
...,...,...
554,0.055921,"(Symptoms: high_fever, Symptoms: phlegm, Sympt..."
555,0.055921,"(Symptoms: chest_pain, Symptoms: phlegm, Sympt..."
556,0.052632,"(Symptoms: malaise, Symptoms: phlegm, Symptoms..."
557,0.052632,"(Symptoms: malaise, Symptoms: chest_pain, Symp..."


In [24]:
apr_rules = association_rules(apr,metric='confidence',min_threshold=0.8)
apr_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,"(Symptoms: malaise, Symptoms: skin_rash)",(Symptoms: headache),0.055921,0.243421,0.052632,0.941176,3.866455,1.0,0.039019,12.861842,0.785279,0.213333,0.922251,0.578696
1,"(Symptoms: headache, Symptoms: skin_rash)",(Symptoms: malaise),0.055921,0.174342,0.052632,0.941176,5.398446,1.0,0.042882,14.036184,0.863023,0.296296,0.928756,0.621532
2,"(Symptoms: headache, Symptoms: skin_rash)",(Symptoms: loss_of_appetite),0.055921,0.266447,0.052632,0.941176,3.532317,1.0,0.037732,12.470395,0.759364,0.195122,0.919810,0.569354
3,"(Symptoms: loss_of_appetite, Symptoms: skin_rash)",(Symptoms: headache),0.055921,0.243421,0.052632,0.941176,3.866455,1.0,0.039019,12.861842,0.785279,0.213333,0.922251,0.578696
4,"(Symptoms: headache, Symptoms: loss_of_appetite)",(Symptoms: skin_rash),0.059211,0.138158,0.052632,0.888889,6.433862,1.0,0.044451,7.756579,0.897727,0.363636,0.871077,0.634921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,"(Symptoms: malaise, Symptoms: phlegm, Symptoms...",(Symptoms: chest_pain),0.052632,0.138158,0.052632,1.000000,7.238095,1.0,0.045360,inf,0.909722,0.380952,1.000000,0.690476
1116,"(Symptoms: chest_pain, Symptoms: phlegm, Sympt...",(Symptoms: malaise),0.055921,0.174342,0.052632,0.941176,5.398446,1.0,0.042882,14.036184,0.863023,0.296296,0.928756,0.621532
1117,"(Symptoms: chest_pain, Symptoms: swelled_lymph...","(Symptoms: malaise, Symptoms: phlegm)",0.055921,0.078947,0.052632,0.941176,11.921569,1.0,0.048217,15.657895,0.970383,0.640000,0.936134,0.803922
1118,"(Symptoms: phlegm, Symptoms: swelled_lymph_nodes)","(Symptoms: malaise, Symptoms: chest_pain)",0.055921,0.082237,0.052632,0.941176,11.444706,1.0,0.048033,15.601974,0.966681,0.615385,0.935906,0.790588


In [27]:
for i,j in apr_rules.iterrows():
    print(j["antecedents"])
    break

frozenset({'Symptoms: malaise', 'Symptoms: skin_rash'})


In [60]:
def normal_pre(users_symps):
    result = []
    for idx,row in apr_rules.iterrows():
        ante = row["antecedents"]
        conse = row["consequents"]
        confidence = row["confidence"]
        jc = jaccard_sml(users_symps,ante)
        
        if jc > 0.1 and jc!=1.0:
           result.append({
            "missing_symptoms":[ante-users_symps],
            "next_symptoms":conse,    
            "confidence":confidence,
           })
        elif jc==1.0:
           result.append({
            "symptoms":conse
           })
    return result        

In [62]:
normal_pre({'Symptoms: skin_rash','Symptoms: red_spots_over_body'})

[{'missing_symptoms': [frozenset({'Symptoms: malaise'})],
  'next_symptoms': frozenset({'Symptoms: headache'}),
  'confidence': 0.9411764705882353},
 {'missing_symptoms': [frozenset({'Symptoms: headache'})],
  'next_symptoms': frozenset({'Symptoms: malaise'}),
  'confidence': 0.9411764705882353},
 {'missing_symptoms': [frozenset({'Symptoms: headache'})],
  'next_symptoms': frozenset({'Symptoms: loss_of_appetite'}),
  'confidence': 0.9411764705882353},
 {'missing_symptoms': [frozenset({'Symptoms: loss_of_appetite'})],
  'next_symptoms': frozenset({'Symptoms: headache'}),
  'confidence': 0.9411764705882353},
 {'missing_symptoms': [frozenset({'Symptoms: malaise'})],
  'next_symptoms': frozenset({'Symptoms: loss_of_appetite'}),
  'confidence': 0.9411764705882353},
 {'missing_symptoms': [frozenset({'Symptoms: loss_of_appetite'})],
  'next_symptoms': frozenset({'Symptoms: malaise'}),
  'confidence': 0.9411764705882353},
 {'missing_symptoms': [frozenset()],
  'next_symptoms': frozenset({'Symp

In [30]:
apr_rules["antecedents"].values

array([frozenset({'Symptoms: malaise', 'Symptoms: skin_rash'}),
       frozenset({'Symptoms: headache', 'Symptoms: skin_rash'}),
       frozenset({'Symptoms: headache', 'Symptoms: skin_rash'}), ...,
       frozenset({'Symptoms: chest_pain', 'Symptoms: swelled_lymph_nodes'}),
       frozenset({'Symptoms: phlegm', 'Symptoms: swelled_lymph_nodes'}),
       frozenset({'Symptoms: abnormal_menstruation'})], dtype=object)

### PRECAUTIONS IMPLEMENTATION 

In [70]:
precautions = pd.read_csv("Data/Sources/symptoms_precautions_updated.csv")
precautions

,Symptom,Precaution
0,anxiety and nervousness,"Maintain sleep schedule, relaxation (yoga/medi..."
1,depression,"Maintain sleep schedule, relaxation (yoga/medi..."
2,insomnia,"Maintain sleep schedule, relaxation (yoga/medi..."
3,emotional symptoms,"Maintain sleep schedule, relaxation (yoga/medi..."
4,hostile behavior,"Maintain sleep schedule, relaxation (yoga/medi..."
...,...,...
320,symptoms of the face,Consult a healthcare provider for proper manag...
321,symptoms of infants,Consult a healthcare provider for proper manag...
322,symptoms of the kidneys,Consult a healthcare provider for proper manag...
323,symptoms of prostate,Consult a healthcare provider for proper manag...


In [71]:
tfidf = TfidfVectorizer(stop_words='english') #removes stopwords (if,or,in etc)
tx = tfidf.fit_transform(precautions['Precaution'])
tx

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 4533 stored elements and shape (325, 97)>

In [106]:
user_sym = {'jaundice'}
s = list(user_sym)
string = " ".join(s)

In [107]:
tr = tfidf.transform([string])
sml = cosine_similarity(tr,tx)
sml

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.24332506, 0.24332506, 0.24332506, 0.24332506, 0.24332506,
        0.24332506, 0.24332506, 0.24332506, 0.24332506, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [108]:
sml.argmax()

30

In [109]:
precautions.loc[30,['Precaution']].values

array(['Drink clean water, avoid unhygienic food, eat light meals, limit oily/spicy food, wash hands, see doctor if blood in stool/jaundice.'],
      dtype=object)

In [9]:
pd.DataFrame(tx.toarray().astype("float"),columns=tfidf.get_feature_names_out())

,alcohol,allergens,avoid,blood,breathing,caffeine,care,chest,clean,compress,...,unhygienic,urgent,urine,use,vision,warm,wash,water,wear,yoga
0,0.265612,0.0,0.0,0.0,0.0,0.318603,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.318603
1,0.265612,0.0,0.0,0.0,0.0,0.318603,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.318603
2,0.265612,0.0,0.0,0.0,0.0,0.318603,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.318603
3,0.265612,0.0,0.0,0.0,0.0,0.318603,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.318603
4,0.265612,0.0,0.0,0.0,0.0,0.318603,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.318603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
321,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
322,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
323,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [78]:
joblib.dump(tx,"Models/precautionsVEC.pkl")

['Models/precautionsVEC.pkl']

### PREDICTIONS AND EVALUATIONS

In [196]:
rules_symptoms = rules[
rules["antecedents"].apply(lambda x: all(i in symptoms for i in x))&
rules["consequents"].apply(lambda x: all(i in symptoms for i in x))
]

In [197]:
rules_diseases = rules[
     rules["antecedents"].apply(lambda x: all( i in symptoms for i in x)) &
     rules["consequents"].apply(lambda x: all(i in diseases for i in x))
]

In [198]:
def jaccard_sml(s1,s2):
    s1 = set(s1)
    s2 = set(s2)
    itrsec = len(s1.intersection(s2))
    union = len(s1.union(s2))
    sml = itrsec / union if union !=0 else 0
    return sml

In [126]:
jaccard_sml(('nasal congestion', 'chest tightness'),('nasal congestion', 'chest tightness'))

1.0

In [400]:
def prediction_diseases(user_symp):
    result_actual = []
    result_missing = []
    for idx,row in rules_diseases.iterrows():
        antecedents = row["antecedents"]
        consequents = row["consequents"]
        confidence = row["confidence"]
        jc = jaccard_sml(user_symp,antecedents)
        if jc > 0.2 and jc!=1.0:
            result_missing.append({
                "symptoms":list(antecedents-user_symp),
                "pos_diseases":list(consequents),
                "confidence":confidence,
            })
        elif jc==1.0:
            result_actual.append({
                "Diseases":consequents,
                "confidence": confidence
            })
            
       # if antecedents.issubset(user_symp) and confidence > 0.5:
       #    result.append([consequents,confidence])
    if len(result_actual)>=1:
        return result_actual
    else:
        return result_missing

In [419]:
de = prediction_diseases({'frequent urination'})
de

[{'Diseases': frozenset({'diseases_pyelonephritis'}),
  'confidence': 0.07638376383763712},
 {'Diseases': frozenset({'diseases_kidney stone'}),
  'confidence': 0.07380073800737942},
 {'Diseases': frozenset({'diseases_prostatitis'}),
  'confidence': 0.04981549815497997},
 {'Diseases': frozenset({'diseases_erectile dysfunction'}),
  'confidence': 0.0455719557195551},
 {'Diseases': frozenset({'diseases_temporary or benign blood in urine'}),
  'confidence': 0.07675276752767193},
 {'Diseases': frozenset({'diseases_urinary tract obstruction'}),
  'confidence': 0.04870848708486853},
 {'Diseases': frozenset({'diseases_cystitis'}),
  'confidence': 0.11512915129150965},
 {'Diseases': frozenset({'diseases_urinary tract infection'}),
  'confidence': 0.07619926199261973},
 {'Diseases': frozenset({'diseases_diabetic ketoacidosis'}),
  'confidence': 0.0605166051660491}]

In [423]:
# import plotly.express as pxs

# # df1 = pd.DataFrame(de).sort_values(by="confidence",ascending=False)
# try:
#     df1 = pd.DataFrame(de).sort_values(by="confidence",ascending=False)
#     l = []
#     l1 = []
#     for i in df1['symptoms']:
#        l.append(re.sub(f"[""''\[\]]+","",str(i)))
#     df1['symptoms'] = l
#     for i in df1['pos_diseases']:
#        l1.append(re.sub(f"[""''\[\]]+","",str(i)))
#     df1['pos_diseases']=l1 
#     for i in df1['symptoms'].value_counts().reset_index().head(3).values:
#         new = df1[df1['symptoms']==i[0]].sort_values(by='confidence',ascending=False).head(5)
#         fig = pxs.pie(new,values='confidence',names='pos_diseases',title=f"Possible Diseases with Symptom: {i[0]}")
#         fig.show()
# except:
#     df1 = pd.DataFrame(de).sort_values(by="confidence",ascending=False).head(3)
#     df1["Diseases"] = df1["Diseases"].apply(lambda x: ", ".join(x) if isinstance(x, frozenset) else str(x))
#     fig = pxs.pie(df1,values="confidence",names="Diseases")
#     fig.show()

In [ ]:
topn = pd.DataFrame(de).sort_values(by='confidence',ascending=False)

In [109]:
topn.head(5).values

array([[list([]), list(['diseases_acute pancreatitis']),
        0.2951289398280788],
       [list([]), list(['diseases_cholecystitis']), 0.2913085004775388],
       [list([]), list(['diseases_hiatal hernia']), 0.20916905444125605],
       [list([]), list(['diseases_gallstone']), 0.20439350525309014],
       [list([]), list(['diseases_esophagitis']), 0.16149999999999762]],
      dtype=object)

In [201]:
def prediction_symptoms(user_symp):
    result = []
    jc_ = []
    for idx,row in rules_symptoms.iterrows():
        antecedents = row["antecedents"]
        consequents = row["consequents"]
        confidence = row["confidence"]
        jc = jaccard_sml(user_symp,antecedents)
        jc_.append(jc)
        if jc > 0.001 and jc!=1.0:
            result.append({
                "symptoms":list(antecedents-user_symp),
                "confidence":confidence,
            })
        elif jc==1.0:
            result.append({
                "symptoms":list(consequents),
                "confidence":confidence,
            })
        
            
        # if antecedents.issubset(user_symp) and confidence > 0.5:
        #    result.append([consequents,confidence])
    output_set = set()
    if len(result)>=1:
        output_df = pd.DataFrame(result).sort_values(by="confidence",ascending=False)["symptoms"].values
        for i in output_df:
            output_set.add(str(i).strip("[]"))
    else:
        pass
    return output_set

In [202]:
res = prediction_symptoms({'hip weakness'})
res

set()

In [422]:
for i in rules_diseases["antecedents"].values:
    print(i)
    

frozenset({'depressive or psychotic symptoms'})
frozenset({'chest tightness'})
frozenset({'anxiety and nervousness'})
frozenset({'irregular heartbeat'})
frozenset({'palpitations'})
frozenset({'shortness of breath'})
frozenset({'dizziness'})
frozenset({'insomnia'})
frozenset({'depression'})
frozenset({'abnormal involuntary movements'})
frozenset({'chest tightness', 'depressive or psychotic symptoms'})
frozenset({'chest tightness', 'abnormal involuntary movements'})
frozenset({'insomnia', 'chest tightness'})
frozenset({'dizziness', 'chest tightness'})
frozenset({'anxiety and nervousness', 'depressive or psychotic symptoms'})
frozenset({'anxiety and nervousness', 'chest tightness'})
frozenset({'dizziness', 'anxiety and nervousness'})
frozenset({'insomnia', 'anxiety and nervousness'})
frozenset({'anxiety and nervousness', 'abnormal involuntary movements'})
frozenset({'irregular heartbeat', 'depressive or psychotic symptoms'})
frozenset({'chest tightness', 'irregular heartbeat'})
frozenset(

In [88]:
for i in rules_symptoms["antecedents"].values:
    print(i)

frozenset({'shortness of breath'})
frozenset({'headache'})
frozenset({'sharp chest pain'})
frozenset({'shortness of breath'})
frozenset({'shortness of breath'})
frozenset({'cough'})
frozenset({'sharp abdominal pain'})
frozenset({'shortness of breath'})
frozenset({'back pain'})
frozenset({'shortness of breath'})
frozenset({'shortness of breath'})
frozenset({'vomiting'})
frozenset({'nausea'})
frozenset({'shortness of breath'})
frozenset({'fever'})
frozenset({'shortness of breath'})
frozenset({'sharp chest pain', 'shortness of breath'})
frozenset({'sharp chest pain', 'headache'})
frozenset({'shortness of breath', 'headache'})
frozenset({'sharp chest pain'})
frozenset({'shortness of breath'})
frozenset({'headache'})
frozenset({'shortness of breath', 'headache'})
frozenset({'shortness of breath', 'cough'})
frozenset({'headache', 'cough'})
frozenset({'shortness of breath'})
frozenset({'headache'})
frozenset({'cough'})
frozenset({'shortness of breath', 'vomiting'})
frozenset({'shortness of br

KeyboardInterrupt: 

In [110]:
rules_diseases["consequents"].value_counts()

consequents
(diseases_hiatal hernia)                         88
(diseases_temporary or benign blood in urine)    84
(diseases_pneumonia)                             79
(diseases_noninfectious gastroenteritis)         78
(diseases_vulvodynia)                            78
                                                 ..
(diseases_aortic valve disease)                   1
(diseases_balanitis)                              1
(diseases_missed abortion)                        1
(diseases_crohn disease)                          1
(diseases_dermatitis due to sun exposure)         1
Name: count, Length: 217, dtype: int64

In [111]:
rules_diseases[rules_diseases["consequents"]=={'diseases_dengue fever'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski


### FINAL REPORT: MODEL IMPLEMENTATION
<li><b>Data Preparation:</b> <br>The dataset was processed using the FP-Growth algorithm with min_support = 0.001 and max_len = 5. This ensures that all possible disease–symptom associations (up to 5 symptoms) are included.</li>

<li><b>Association Rule:</b><br>Association rules were generated with evaluaion metric as confidence and min_threshold 0.001.<br>
Rules with high confidence were prioritized, as they indicate stronger symptom–disease relationships.</li>

<li><b>Jaccard Similarity:</b><br>jaccard index (initially set to 0.3) used to handle cases where users provide insufficient symptoms.<br>
Jaccard threshold is adjustable to improve recommendations and model performance.
</li>
<li><b>Precautions:</b><br>Users precautions were transformed using TfidfVectorizer(with stopwords(English) removal).<br>
Cosine similarity to recommend relevant precautions irrespective of diseases.
</li>
<li><b>Severity:</b><br>Recommended symptoms were mapped to severity dataset to predict severity level.<br>
The overall severity level was calculated as the average severity score across all symptoms.</li>